In [1]:
from caffe import layers as L, params as P
import caffe
from pylab import *
from tqdm import tqdm_notebook as tqdm

caffe.set_mode_gpu()
caffe.set_device(0)

In [2]:
class CaffeSolverDecorator:
    
    def __init__(self, name):
        root = 'proto'
        self.name = name
        self.solver_path = os.path.join(root, self.name, '_solver.prototxt')
        self.train_path = os.path.join(root, self.name, '_train.prototxt')
        self.test_path = os.path.join(root, self.name, '_test.prototxt')
        self.solver = None
    
    def create_solver_conf(self, optimize_type, lr, need_snapshot=False):
        s = caffe.proto.caffe_pb2.SolverParameter()
        
        s.train_net = self.train_path
        
        s.test_net.append(self.test_path)
        
        s.type = optimize_type
        s.base_lr = lr
        
        s.lr_policy = 'inv'
        s.gamma = 0.0001
        s.weight_decay = 0.0005
        
        if optimize_type.__eq__('SGD') or optimize_type.__eq__('Adam'):
            s.momentum = 0.9
            
        if optimize_type.__eq__('Adam'):
            s.momentum2 = 0.999
            s.delta = 10e-8
            
        if optimize_type.__eq__('RMSProp'):
            s.rms_decay = 0.99
            
        s.solver_mode = caffe.proto.caffe_pb2.SolverParameter.GPU
        
        with open(self.solver_path, 'w') as f:
            f.write(str(s))
            
        self.solver = caffe.get_solver(self.solver_path)
        
    def show_net_model(self):
        data =  [(k, v.data.shape) for k, v in solver.net.blobs.items()]  
        print(data)

    def start_solver(solver, epoch=10_000, batch_size=40, test_interval=500, test_iter=100, train_interval=100):
        train_loss = zeros(int(epoch / train_interval))
        train_acc = zeros(int(epoch / train_interval))
        test_acc = zeros(int(epoch / test_interval))
        test_loss = zeros(int(epoch / test_interval))


        for it in tqdm(range(epoch)):
            solver.step(1)

            if it % train_interval == 0:
                print(f"###### Iterration #{it}")
                train_loss[it // train_interval] = solver.net.blobs['loss'].data
                train_acc[it // train_interval] = solver.net.blobs['accuracy_top1'].data
                print(f"Train net output: accuracy = {train_acc[it // train_interval]}, loss = {train_loss[it // train_interval]}")

            if it % test_interval == 0:
                correct = 0
                for test_it in range(test_iter):
                    solver.test_nets[0].forward()

                    correct += sum(solver.test_nets[0].blobs['score'].data.argmax(axis=1)
                                   == solver.test_nets[0].blobs['label'].data)

                test_acc[it // test_interval] = correct / (test_iter * batch_size)

            test_loss[it // test_interval] = solver.test_nets[0].blobs['loss'].data
            print(f"Test net output: accuracy = {test_acc[it // test_interval]}, loss = {test_loss[it // test_interval]}")
                
        self.show_plots("Train", train_loss, train_acc, train_interval)
        self.show_plots("Test", test_loss, test_acc, test_interval)
        return train_loss, train_acc, train_interval, test_loss, test_acc, test_interval
    
    def show_plots(what_see, test_loss, test_acc, test_interval):
        _, ax1 = subplots()
        ax2 = ax1.twinx()
        ax1.plot(test_interval * arange(len(test_loss)), test_loss)
        ax2.plot(test_interval * arange(len(test_acc)), test_acc, 'r')
        ax1.set_xlabel('iteration')
        ax1.set_ylabel(what_see + ' loss')
        ax2.set_ylabel(what_see + ' accuracy')
        ax2.set_title(what_see + ' Accuracy: {:.2f}'.format(test_acc[-1]))

In [17]:
class CaffeNet:
    
    def __init__(self, name):
        self.name = name
        self.net_train = caffe.NetSpec()
        self.net_test = caffe.NetSpec()
        self.layers_creator = CaffeLayersCreator()
        self.is_created = False
        self.net_train_top_layer = None
        self.net_test_top_layer = None
        
        self.conv_counter = 1
        self.relu_counter = 1
        self.pool_counter = 1
        self.batch_counter = 1
        self.fc_counter = 1
        
    # Hidden layers = in_place attr
    def set_global_params(self, batch_sizes, weight_params, bias_params, image_params, count_of_classes, hidden_layers=True):
        self.train_batch_size = batch_sizes['train']
        self.test_batch_size = batch_sizes['test']
        self.weight_params = weight_params
        self.image_params = image_params
        self.count_of_classes = count_of_classes
        self.hidden_layers = hidden_layers
        
        if self.is_created:
            print('Call build before you will try start the train (need to update global params)')
        
    def l_image_data(self, image_params=None):
        if image_params is None:
            image_params = self.image_params
        
        train_set = self.image_params['train']
        test_set = self.image_params['test']
        self.net_train['data'], self.net_train['label'] = self.layers_creator.l_image_data_create(self.train_batch_size, train_set, image_params)
        self.net_test['data'], self.net_test['label'] = self.layers_creator.l_image_data_create(self.test_batch_size, test_set, image_params)
                
        self.net_train_top_layer = self.net_train['data']
        self.net_test_top_layer = self.net_test['data']
        
    def l_conv(self, kernel, filter_count, weight_filter=None, padding=1):
        if weight_filter is None:
            weight_filter = self.weight_params
        layer_name = 'conv' + str(self.conv_counter)
        self.net_train[layer_name] = self.layers_creator.l_conv_create(self.net_train_top_layer, kernel, filter_count, weight_filter, padding)
        self.net_test[layer_name] = self.layers_creator.l_conv_create(self.net_test_top_layer, kernel, filter_count, weight_filter, padding)
        self.conv_counter += 1
                
        self.net_train_top_layer = self.net_train[layer_name]
        self.net_test_top_layer = self.net_test[layer_name]
        
        
    def l_pool(self, kernel_size, stride, pool=P.Pooling.MAX):
        layer_name = 'pool' + str(self.pool_counter)
        self.net_train[layer_name] = self.layers_creator.l_pool_create(self.net_train_top_layer, kernel_size, stride, pool)
        self.net_test[layer_name] = self.layers_creator.l_pool_create(self.net_test_top_layer,  kernel_size, stride, pool)
        self.pool_counter += 1
        
        self.net_train_top_layer = self.net_train[layer_name]
        self.net_test_top_layer = self.net_test[layer_name]
    
    def l_batch(self):
        layer_name = 'batch' + str(self.batch_counter)
        self.net_train[layer_name] = self.layers_creator.l_batch_create(self.net_train_top_layer)
        self.net_test[layer_name] = self.layers_creator.l_batch_create(self.net_test_top_layer)
        self.batch_counter += 1
        print(self.net_train.to_proto())
        print(self.net_test.to_proto())
        
        self.net_train_top_layer = self.net_train[layer_name]
        self.net_test_top_layer = self.net_test[layer_name]
        
    def l_relu(self, hidden=None):
        if hidden is None:
            hidden = self.hidden_layers
        layer_name = 'relu' + str(self.relu_counter)
        self.net_train[layer_name] = self.layers_creator.l_relu_create(self.net_train_top_layer, hidden)
        self.net_test[layer_name] = self.layers_creator.l_relu_create(self.net_test_top_layer, hidden)
        self.relu_counter += 1
        
        self.net_train_top_layer = self.net_train[layer_name]
        self.net_test_top_layer = self.net_test[layer_name]
        
    def l_fc(self, num_output):
        if weight_filter is None:
            weight_filter = self.weight_params
        layer_name = 'dense' + str(self.fc_counter)
        self.net_train[layer_name] = self.layers_creator.l_fc_create(self.net_train_top_layer, num_output, weight_filter)
        self.net_test[layer_name] = self.layers_creator.l_fc_create(self.net_test_top_layer, num_output, weight_filter)
        self.fc_counter += 1
        
        self.net_train_top_layer = self.net_train[layer_name]
        self.net_test_top_layer = self.net_test[layer_name]

    def l_score(self):
        self.net_train['score'] = self.layers_creator.l_score_create(self.net_train_top_layer, num_output=self.count_of_classes, weight_filter=self.weight_params)
        self.net_test['score'] = self.layers_creator.l_score_create(self.net_test_top_layer, num_output=self.count_of_classes, weight_filter=self.weight_params)
        
    def l_loss(self):
        self.net_train['loss'] = self.layers_craetor.l_loss_create(self.net_train['score'], self.net_train['label'])
        self.net_test['loss'] = self.layers_craetor.l_loss_create(self.net_train['score'], self.net_train['label'])
        
    def l_acc(self):
        self.net_train['accuracy'] = self.layers_craetor.l_acc_create(self.net_train['score'], self.net_train['label'])
        self.net_test['accuracy'] = self.layers_craetor.l_acc_create(self.net_train['score'], self.net_train['label'])
        
    def l_conv_batch_relu(self, kernel, filter_count, weight_filter=None, padding=1):
        if weight_filter is None:
            weight_filter = self.weight_params
        self.l_conv(kernel, filter_count, weight_filter, padding)
        self.l_batch()
        self.l_relu()
        
    def l_conv_relu(self,  kernel, filter_count, weight_filter=None, padding=1):
        if weight_filter is None:
            weight_filter = self.weight_params
        self.l_conv(kernel, filter_count, weight_filter, padding)
        self.l_relu()
        
    def l_fc_relu(self, num_output, weight_filter=None):
        if weight_filter is None:
            weight_filter = self.weight_params
        self.l_fc(num_output, weight_filter)
        self.l_relu()
        
    def l_score_loss_acc(self):
        self.l_score()
        self.l_loss()
        self.l_acc()
        
    def build(self):
        train_name = os.path.join('proto',  self.name + '_train.prototxt') 
        test_name = os.path.join('proto',  self.name + '_test.prototxt') 
        
        with open(train_name, 'w') as f:
            f.write(str(self.net_train.to_proto()))

        with open(test_name, 'w') as f:
            f.write(str(self.net_test.to_proto()))
            
        self.conv_counter = 1
        self.relu_counter = 1
        self.pool_counter = 1
        self.batch_counter = 1
        self.fc_counter = 1
        
        print("Build was successfull")

In [18]:
class CaffeLayersCreator:
    
    def l_image_data_create(self, batch, _source, image_params):
        return L.ImageData(
            batch_size = batch,
            source = _source,
            is_color = image_params['is_color'],
            new_height = image_params['new_height'],
            new_width = image_params['new_width'],
            shuffle = image_params['shuffle'],
            ntop=2)
    
    def l_conv_create(self, top, kernel, filter_count, weight_filter, padding):
        return L.Convolution(
            top,
            kernel_size=kernel,
            pad=padding,
            num_output=filter_count,
            weight_filler=weight_filter
        )
    
    def l_pool_create(self, top, kernel, shift, pool_strategy):
        return L.Pooling(
            top,
            kernel_size=kernel,
            stride=shift,
            pool=pool_strategy
        )
        
    def l_batch_create(self, top):
        return L.BatchNorm(top)
        
    def l_relu_create(self, top, is_hidden):
        return L.ReLU(top, is_hidden)
        
    def l_fc_create(self, top, weight_filter):
        return L.InnerProduct(top, weight_filler=weight_filter)
        
    def l_score_create(self, top, weight_filter):
        return l_fc_create(top, weight_filter)
        
    def l_loss_create(self, score, label):
        return L.SoftmaxWithLoss(score, label)
        
    def l_acc_create(self, score, label):
        return L.Accuracy(score, label)

In [19]:
net = CaffeNet('improved_vgg10')
net.set_global_params(
    batch_sizes=dict(train=40, test=40),
    weight_params=dict(type='xavier'),
    bias_params=dict(type='constant', value=0.1),
    image_params=dict(train='resources/train.txt', test='resources/test.txt', is_color=False, new_height=200, new_width=200, shuffle=True),
    count_of_classes=11)

net.l_image_data()

net.l_conv_batch_relu(3, 32)
net.l_conv_batch_relu(3, 32)
net.l_pool(2, 2)

net.l_conv_batch_relu(3, 64)
net.l_conv_batch_relu(3, 64)
net.l_pool(2, 2)

net.l_conv_batch_relu(3, 96)
net.l_conv_batch_relu(3, 96)
net.l_conv_batch_relu(3, 96)
net.l_pool(2, 2)

net.l_conv_batch_relu(3, 128)
net.l_conv_batch_relu(3, 128)
net.l_pool(2, 2)

net.l_fc_relu(2000)
net.l_fc_relu(1000)
net.l_fc_relu(500)

net.l_score_loss_acc()

layer {
  name: "data"
  type: "ImageData"
  top: "data"
  top: "label"
  image_data_param {
    source: "resources/train.txt"
    batch_size: 40
    shuffle: true
    new_height: 200
    new_width: 200
    is_color: false
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 32
    pad: 1
    kernel_size: 3
    weight_filler {
      type: "xavier"
    }
  }
}
layer {
  name: "batch1"
  type: "BatchNorm"
  bottom: "conv1"
  top: "batch1"
}

layer {
  name: "data"
  type: "ImageData"
  top: "data"
  top: "label"
  image_data_param {
    source: "resources/test.txt"
    batch_size: 40
    shuffle: true
    new_height: 200
    new_width: 200
    is_color: false
  }
}
layer {
  name: "conv1"
  type: "Convolution"
  bottom: "data"
  top: "conv1"
  convolution_param {
    num_output: 32
    pad: 1
    kernel_size: 3
    weight_filler {
      type: "xavier"
    }
  }
}
layer {
  name: "batch1"
  type: "BatchNorm"
  bottom: "c

TypeError: ReLU input 1 is not a Top (type is <class 'bool'>)